# Optimization-based feature selection with adaptive instance sampling

In [143]:
import pandas as pd
import numpy as np

In [144]:
df = pd.read_excel("hypothetical data.xlsx",sheet_name="static")
m = df.shape[0]
df.head()

,y,x1,x2,x3,x4,c
0,154,8,20,4,42,2
1,196,9,22,5,49,2
2,198,10,19,4,57,1
3,178,9,19,3,47,2
4,157,9,18,4,65,2


In [160]:
import math
c = len(df.c.unique())

def func_s(df,T_Independent_Variables):
    features_list = list(T_Independent_Variables.columns)
    i_list = list(df['c'].unique())
    s = {}
    S = {}
    p = {}
    p_IT = {}
    q = {}
    I = {}
    E = {}
    Gain = {}
    # calculate s, S
    for a in features_list:
        for j in T_Independent_Variables[a].unique():
            S[(j,a)] = 0
            q[j,a] = 0
            for i in i_list:
                filtered_df = df.loc[(df[a] == j) & (df['c'] == i)]
                count = filtered_df[a].count()
                s[(i,j,a)] = count
                S[(j,a)] += count

            q[j,a] = S[j,a]/m
    # print("q",q)
    # calculate p, I
    for a in features_list:
        for j in T_Independent_Variables[a].unique():
            I[j,a] = 0
            for i in i_list:
                p[i,j,a] = s[i,j,a]/S[j,a]
                if p[i,j,a]>0:
                    I[(j,a)] += (-1) * p[i,j,a] * math.log2(p[i,j,a])

    # calculate IT
    IT = 0
    for i in i_list:
        for a in features_list:
            p_IT[i,a] = 0
            for j in T_Independent_Variables[a].unique():
                p_IT[i,a] += s[i,j,a]/m
            IT += (-1) * p_IT[i,a] * math.log2(p_IT[i,a])

    # calculate E
    for a in features_list:
        E[a] = 0
        for j in T_Independent_Variables[a].unique():
            E[a] += q[j,a] * I[j,a]
        Gain[a] = IT - E[a]
    # sorting
    sorted_E = {k: v for k, v in sorted(E.items(), key=lambda item: item[1])}
    print(sorted_E)
    return Gain




In [161]:
T_Independent_Variables = df.drop(['y', 'c'], axis=1)

features_list = list(T_Independent_Variables.columns)

In [162]:
func_s(df,T_Independent_Variables)

{'x4': 0.6329331487002061, 'x2': 0.9138894366941896, 'x1': 0.9399461579865102, 'x3': 0.9646233349152583}


{'x1': 3.0588996090247296,
 'x2': 3.08495633031705,
 'x3': 3.0342224320959814,
 'x4': 3.3659126183110333}

In [114]:
df.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'c'], dtype='object')

In [115]:
df.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'c'], dtype='object')

In [163]:
df[features_list[0]].unique()


array([ 8,  9, 10, 11,  6,  7, 12], dtype=int64)